In [21]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

import pickle

In [22]:
df = pd.read_csv('Prosv84_26_02_2021_1_year.csv', sep=';')

In [23]:
df.drop(['КоэффициентМинимальнойПоставкиИзлишков', 'БазоваяЕдиницаИзмеренияАйриса'], axis=1, inplace=True)

In [24]:
df.dropna(subset=['Артикул', 'СтатусТовараАйриса', 'ЕдиницаИзмерения', 'ОптоваяФасовкаАйрис', 
                  'КоэффициентЕдиницы', 'Цена', 'ЗакупочнаяЦена', 'ОсновнойПоставщик'], inplace=True)

In [25]:
ind_for_drop = df[(df['СредниеПродажи'].isna() & df['КоличествоСделок'] > 0)
                  | (df['ЗакупкаЗаПериод'] < 0) | (df['ПроданоВсего'] < 0)
                  | (df['СтатусТовараАйриса'] == 'Исключенный')].index
df_1 = df.drop(list(ind_for_drop))  #### удалить строчки, в ктр средние продажи = 0, а кол-во сделок> 0 и т.д.

In [26]:
df_1[['КодХарактеристики', 'Артикул']] = df_1[['КодХарактеристики', 'Артикул']].astype(dtype='Int32')

In [27]:
df_1.loc[df['МинимальныйСтраховойЗапас'] < 1, 'МинимальныйСтраховойЗапас'] = 0.001 # приводим все мин ктр меньше 1 к одному числу

In [28]:
# Удаляем значения мин-макс по кол-ву менее 10!! (не будем на них обучаться!)
#df_1.groupby('ЗначениеТочкиЗаказа').nunique()

df_1['freq_z']=df.groupby(by='ЗначениеТочкиЗаказа')['ЗначениеТочкиЗаказа'].transform('count')
df_1['freq_s']=df.groupby(by='МинимальныйСтраховойЗапас')['МинимальныйСтраховойЗапас'].transform('count')

df_1 = df_1[(df_1['freq_z'] >= 10) & (df_1['freq_s'] >= 10)].drop(['freq_s', 'freq_z'],  axis=1)
#df_1.groupby(['Name','City']).filter(lambda x: sum(x['Age']) > 30)

In [29]:
X = df_1.drop(['МинимальныйСтраховойЗапас', 'ЗначениеТочкиЗаказа'], axis=1)
y = df_1.loc[:, ['МинимальныйСтраховойЗапас', 'ЗначениеТочкиЗаказа']]   # таргет

In [30]:
y_need = y[(y['ЗначениеТочкиЗаказа'] == 0)]  # убрал из выражения:y['МинимальныйСтраховойЗапас'] == 0) & ....
X_need = X.loc[list(y_need.index)]

y_filter = y.drop(list(y_need.index))      # формируем значения МИН и МАХа на ктр будем обучаться

X_filter = X.drop(list(y_need.index))      # формируем соответствующие данные на ктр будем обучаться

In [31]:
y_filter_min = y_filter['МинимальныйСтраховойЗапас']
y_filter_max = y_filter['ЗначениеТочкиЗаказа']

In [32]:
# load
data_transformer_1 = pickle.load(open('data_transformer_1.pk', 'rb'))
data_transformer_2 = pickle.load(open('data_transformer_2.pk', 'rb'))

In [33]:
# load:
with open('min_name.pkl', 'rb') as f:
    grid_min_s = pickle.load(f)
    
with open('max_name.pkl', 'rb') as f:
    grid_max_s = pickle.load(f)

In [35]:
y_need = y[(y['ЗначениеТочкиЗаказа'] == 0)]  # убрал из выражения:y['МинимальныйСтраховойЗапас'] == 0) & ....
X_need = X.loc[list(y_need.index)]

y_filter = y.drop(list(y_need.index))      # формируем значения МИН и МАХа на ктр будем обучаться

X_filter = X.drop(list(y_need.index))      # формируем соответствующие данные на ктр будем обучаться

In [36]:
y_filter_min = y_filter['МинимальныйСтраховойЗапас']
y_filter_max = y_filter['ЗначениеТочкиЗаказа']

In [37]:
le_min = LabelEncoder()  #Делаем label encoding
le_max = LabelEncoder()

y_filter_min_le = le_min.fit_transform(y_filter_min)
y_filter_max_le = le_max.fit_transform(y_filter_max)

In [38]:

# load
#le_min = pickle.load(open('le_min.pk', 'rb'))
#le_max = pickle.load(open('le_max.pk', 'rb'))

In [39]:
# Если не вставим эту ячейку, то будет рассогласование (например 1036 и 1031)
#X_train_min_trans = data_transformer_1.fit_transform(X_train_min) # далее обрабатываем трансформированные значения!!
#X_train_max_trans = data_transformer_2.fit_transform(X_train_max)

In [41]:
X_train_min, X_test_min, y_train_min, y_test_min = train_test_split(
    X_filter, y_filter_min_le, test_size=0.33, stratify=y_filter_min_le, random_state=42
)
X_train_max, X_test_max, y_train_max, y_test_max = train_test_split(
    X_filter, y_filter_max_le, test_size=0.33, stratify=y_filter_max_le, random_state=42
)

In [42]:
X_test_min_trans = data_transformer_1.transform(X_test_min) # не забываем, что используем уже обученную модель на новых быстрых данных
X_test_max_trans = data_transformer_2.transform(X_test_max)

In [43]:
X_test_max_trans

array([[ 0.626,  3.754,  2.   , ...,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   , ...,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   , ...,  0.   ,  0.   ,  0.   ],
       ...,
       [ 0.625,  5.   ,  5.   , ...,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   , ...,  0.   ,  0.   ,  0.   ],
       [ 2.27 , 18.161, 16.   , ...,  0.   ,  0.   ,  0.   ]])

In [44]:
pd.DataFrame(X_test_min_trans)

,0,1,2,3,4,5,6,7,8,9,...,1021,1022,1023,1024,1025,1026,1027,1028,1029,1030
0,0.000,0.000,0.0,0.0,19.00,0.000000,0.000,0.000,0.000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.833,25.532,12.0,8.0,394.00,46.800156,21.996,14.664,722.202,306.384,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.574,6.883,6.0,3.0,496.00,3.950842,3.444,1.722,284.704,41.298,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.072,1.000,1.0,1.0,2.00,0.072000,0.072,0.072,0.144,1.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000,0.000,0.0,0.0,17.00,0.000000,0.000,0.000,0.000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8343,0.000,0.000,0.0,0.0,19.00,0.000000,0.000,0.000,0.000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8344,0.054,0.645,1.0,1.0,15.00,0.034830,0.054,0.054,0.810,0.645,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8345,0.000,0.000,0.0,0.0,24.00,0.000000,0.000,0.000,0.000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8346,0.000,0.000,0.0,0.0,267.52,0.000000,0.000,0.000,0.000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [45]:
pred_min = grid_min_s.predict(X_test_min_trans)
pred_max = grid_max_s.predict(X_test_max_trans)

In [46]:
frame_min = [
    pd.DataFrame(pred_min, columns=['predict']), 
    pd.DataFrame(y_test_min, columns=['Минимальный остаток']).reset_index(drop=True), X_test_min[['ОптоваяФасовкаАйрис']].reset_index(drop=True)
]
frame_max = [
    pd.DataFrame(pred_max, columns=['predict']), 
    pd.DataFrame(y_test_max, columns=['Нормативный остаток']).reset_index(drop=True), X_test_max['ОптоваяФасовкаАйрис'].reset_index(drop=True)
]


In [47]:
df3_min = pd.concat(frame_min, axis=1)
df3_max = pd.concat(frame_max, axis=1)

In [48]:
accuracy_score(df3_min['Минимальный остаток'], df3_min['predict'])

0.7823430761859128

In [49]:
#Делаем label encoding
#y_filter_mddin_le = le_min.transform(y_filter_min)
#y_filter_max_le = le_max.transform(y_filter_max)

In [50]:
accuracy_score(df3_max['Нормативный остаток'], df3_max['predict'])

0.6511739338763776

In [52]:
#X_train_min, X_test_min, y_train_min, y_test_min = train_test_split(
   # X_filter, y_filter_min_le, test_size=0.33, stratify=y_filter_min_le, random_state=42
#)
X_train_max, X_test_max, y_train_max, y_test_max = train_test_split(
    X_filter, y_filter_max_le, test_size=0.33, stratify=y_filter_max_le, random_state=42
)

In [53]:
X_test_min

,НомерСтроки,Артикул,Код,КодХарактеристики,ГруппаНоменклатуры,Номенклатура,ЕдиницаИзмерения,ОптоваяФасовкаАйрис,КоэффициентЕдиницы,СредниеПродажи,...,ПроданоВсего,КоличествоСделок,КоличествоПериодовПродаж,КоличествоДнейНаСкладе,ЗакупкаЗаПериод,СтатусТовараАйриса,КатегорияТовара,КатегорияТовараАйриса,ЗакупочнаяЦена,ОсновнойПоставщик
41960,41961,7705698,02/00016418,1,Астра,"Подвеска-спираль FIN0840-50, d-50мм, 1шт/уп",упак,1.0,1.0,0.000,...,0.000,0,0,366,19.00,Удалить,B,NaN,19.20,Китай Microstar
8386,8387,7722477,Б/00077236,8,"Пряжа ""ALIZE""","Пряжа Alize ""Baby Best"" 100г 240м (10% бамбук,...",шт,5.0,1.0,1.833,...,25.532,12,8,366,394.00,Регулярный,A,A,69.32,Турция Alize
3053,3054,7713421,02/00034971,1,Пайетки Астра,"Пайетки плоские 3 мм, ""Астра"" 10 г.",шт,10.0,1.0,0.574,...,6.883,6,3,289,496.00,Регулярный,A,A,19.80,Китай Sheng Tong Fashion
40574,40575,551749,Б/00068554,<NA>,Наполнение для стенда,131-11-030 Кабошоны (жемчуг) White 3мм. 40 ш...,упак,1.0,1.0,0.072,...,1.000,1,1,366,2.00,Фасовка,C,C,70.28,ПРЕЦИОСА-ЦЕНТР ООО
45102,45103,183034,00000016762,82,"Манжеты, подвязы",Манжет акрил 7см*10см,пар,1.0,1.0,0.000,...,0.000,0,0,366,17.00,Регулярный,C,C,35.70,ЛАДОГА ООО
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43728,43729,7712716,02/00032347,1,Астра,"Основа для серег 54-P7681, 18*7mm, 2шт/упак",упак,1.0,1.0,0.000,...,0.000,0,0,366,19.00,Удалить,NaN,NaN,38.98,Китай Microstar
45112,45113,183034,00000016762,101,"Манжеты, подвязы",Манжет акрил 7см*10см,пар,1.0,1.0,0.054,...,0.645,1,1,341,15.00,Регулярный,C,B,35.70,ЛАДОГА ООО
293,294,612875,Б/00106406,2,"Заколки, броши, булавки декоративные разные",Украшение для обуви и сумок «Ромашка»,шт,1.0,1.0,0.000,...,0.000,0,0,0,24.00,Удалить,C,NaN,23.65,РУСПВХ ООО
33235,33236,422002,00000037514,62,Косая бейка атласная,Косая бейка 15мм*132м 0000-1500,м,132.0,1.0,0.000,...,0.000,0,0,0,267.52,Регулярный,B,C,0.72,Китай MH Industry
